**1**. (25 points)

- Download the data from the URL given into a `pandas` DataFrame (5 points)
```
https://gist.github.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv
```
- Your objective is to predict the `mpg` of a car from `hp`, `wt` and `am`
    - Use the last 10 rows as test data and the rest as training data (5 points)
    - Train a multiple linear regression model on the training data (10 points)
    - Evaluate the mean squared error on the test data (5 points)
    
You may **only** use the following class from `sklearn`  (default parameters are fine)

- `sklearn.linear_model.LinearRegression`

In particular, splitting into test and train data and calculaiton of mean squared error should not use `sklearn`

In [143]:
import pandas as pd
import numpy as np
#Import data
df = pd.read_csv('https://gist.github.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv')

In [144]:
df.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [145]:
# Get test data and training data
training = df.iloc[:-10]
test = df.iloc[-10:]

In [146]:
print(len(training), len(test), len(df))

22 10 32


In [147]:
# Get data into proper form for regression
train_x_df = training[['wt','hp', 'am']]
train_y_df = training['mpg']
test_x_df = test[['wt','hp', 'am']]
test_y_df = test[['mpg']]

In [148]:
import sklearn.linear_model as lm
# Fit model on training data
model = lm.LinearRegression().fit(train_x_df, train_y_df)

In [149]:
# Predict using model on testing data
preds = model.predict(test_x_df)

In [150]:
# Calculate the mean square error
np.mean(np.square(list(test_y_df['mpg']) - preds))

5.11965278142159

**3**. (25 points)

We have provided an SQLite3 database in `data/pets.db` with 3 tables `dog`, `treat` and `dog_treat`. The `dog_treat` table is a linker table showing which dog ate which treat. 

- Show a table of ALL dogs and the treats with calories that they ate with column names `dog`, `treat`, `calorie`. A dog that did not eat any treats should still be present in the table (15 points)
- Using a common table expression, show a table with two columns `dog` and `total_calories` where only dogs that have eaten more than 500 calories are displayed (5 points)

In [151]:
import sqlite3
# Create database connection
%load_ext sql
%sql sqlite:///data/pets.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @data/pets.db'

In [152]:
%%sql

SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///data/pets.db
Done.


type,name,tbl_name,rootpage,sql
table,dog,dog,2,"CREATE TABLE dog ( ""index"" BIGINT, dog_id BIGINT, name TEXT, age BIGINT)"
table,treat,treat,4,"CREATE TABLE treat ( ""index"" BIGINT, treat_id BIGINT, name TEXT, calories BIGINT)"
table,dog_treat,dog_treat,6,"CREATE TABLE dog_treat ( ""index"" BIGINT, treat_id BIGINT, dog_id BIGINT)"


Question 1

In [175]:
%%sql

SELECT dog.name as dog, treat.name as treat, treat.calories as calorie
FROM dog
LEFT OUTER JOIN dog_treat on dog.dog_id = dog_treat.dog_id
LEFT OUTER JOIN treat ON dog_treat.treat_id = treat.treat_id

 * sqlite:///data/pets.db
Done.


dog,treat,calorie
Fido,Chewies,100
Fido,Bones,500
Roger,Bones,500
Woof,Bones,500
Woof,Raw Meat,450
Lassie,None,None


Question 2

In [176]:
%%sql

WITH foodtable (dog, total_calories)
AS 
(
    SELECT dog.name as dog, SUM(treat.calories) as total_calories
    FROM dog
    LEFT OUTER JOIN dog_treat on dog.dog_id = dog_treat.dog_id
    LEFT OUTER JOIN treat ON dog_treat.treat_id = treat.treat_id
    GROUP BY dog

)

SELECT dog, total_calories
FROM foodtable
WHERE total_calories>500

 * sqlite:///data/pets.db
Done.


dog,total_calories
Fido,600
Woof,950


**4**. (40 points)

You want to evaluate whether a liner, quadratic or cubic polynomial is the best model for a set of data using leave-one-out cross-validation (LOOCV) and the mean squared error as evaluation metric. 

- Write a function named `loocv` that takes the predictor variable `x`, the outcome variable `y`, a list of of degrees of polynomial models to be evaluated, and an evaluation function and returns the best model found by LOOCV. For example, you would call the function like this `loocv(x, y, [1,2,3], mse)` where `mse` is of course a function that returns the mean squared error. (30 points)
- Write the `mse` function to provide to the LOOCV routine (5 points)
- Use the `llocv` function to find the best polynomial model for the data provided (5 points)

Notes

- Use the `x` and `y` variables provided
- Do not use any packages except for the standard library and `numpy`
- Code snippets for fitting and evaluation of polynomials is provided

In [154]:
import numpy as np

In [177]:
x = np.load('data/x.npy')
y = np.load('data/y.npy')

In [156]:
coeffs = np.polyfit(x, y, 2)
ypred = np.polyval(coeffs, x)

In [179]:
def mse(y, pred):
    mse = np.mean(np.square(y-pred))
    return(mse)

def loocv(x, y, models, mse):
    # Save the errors for each model
    model_errors = []
    for i in models: # Test out each model
        # Store the errors for each fold of cross validation to average later
        cross_val_errors = []
        for j in x: # Fit the model once on each fold
            subset_x = [d for index,d in enumerate(x) if index!=j]  # Get all but the ith data point for the fold
            subset_y = [c for index,c in enumerate(y) if index!=j] 
            coeffs = np.polyfit(subset_x,subset_y,i)
            cross_val_errors.append(mse(y, np.polyval(coeffs, x))) # Save the error of the model for that fold
        model_errors.append(sum(cross_val_errors)/len(cross_val_errors)) # Find the average error of the model across all folds
    index = model_errors.index(min(model_errors)) # Find the index of the best model so it can be returned
    return(models[index], model_errors[index])
    

In [180]:
loocv(x, y, [1,2,3], mse)

(2, 0.016290109581223112)